In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
from collections import Counter

import folium
import warnings
warnings.filterwarnings('ignore')
import ast
import math

## Parking Violation Map

In [2]:
par_viol = pd.read_csv("Parking_Violations_Issued_-_Fiscal_Year_2022.csv")
par_viol.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Vehicle Expiration Date,...,House Number,Street Name,Date First Observed,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Violation Post Code,Violation Description
0,1457617912,JEB5683,NY,PAS,6/25/2021,40,VAN,FORD,P,20221004,...,100,NORTH PORTLAND AVE,0,BBBBBBB,ALL,ALL,GY,0.0,NaN,NaN
1,1457617924,JAN2986,NY,PAS,6/25/2021,20,SUBN,DODGE,P,20221203,...,398,AUBURN PLACE,20210625,BBBBBBB,ALL,ALL,BLU,0.0,NaN,NaN
2,1457622427,FJH6630,TX,PAS,6/17/2021,98,SDN,AUDI,P,0,...,170,SOUTH PORTLAND AVE,0,BBBBBBB,ALL,ALL,WHITE,0.0,NaN,NaN
3,1457638629,RD1Y5N,MO,PAS,6/16/2021,98,SDN,TOYOT,P,0,...,22,JEFFERSON ST,0,BBBBBBB,ALL,ALL,TAN,0.0,NaN,NaN
4,1457639580,T503814C,NY,OMT,7/4/2021,40,TAXI,HONDA,P,20210831,...,25,STANHOPE ST,0,BBBBBBB,ALL,ALL,WHI,0.0,NaN,NaN


we get the address of where parking violation happened with house number and street name.\
and then convert the address into longitute and latitute\
after that we are mapping the spot on the map

In [3]:
address = par_viol['House Number']+ ' '+par_viol['Street Name']

In [4]:
len(address)

1048575

cleaning the data by removing all the na 

In [5]:
ad = address.fillna(np.nan).dropna()

In [6]:
len(ad)

616708

In [7]:
l = sorted(list(ad), reverse = True)

sorted the data reversely, we are taking the most frequent data

In [8]:
from collections import Counter
f = Counter(l)
k = sorted(f.items(), key=lambda x: x[1], reverse=True)

### taking where car colission happened more than 100 times

by checking the number, we have located the first 166 data are the ones appearing more than 100 times

In [10]:
add1=  k[0:166]
add1

[('W Broadway', 714),
 ('E Broadway', 688),
 ('W Madison Ave', 532),
 ('W 3rd Ave', 515),
 ('E 3rd Ave', 456),
 ('E 5th Ave', 425),
 ('E 2nd Ave', 420),
 ('2655 Richmond Ave', 367),
 ('E Park Ave', 349),
 ('W 6th Ave', 344),
 ('S Queens Blvd', 343),
 ('E West St', 334),
 ('S Surf Ave', 326),
 ('W Park Ave', 312),
 ('S W 49th St', 306),
 ('E Lexington Ave', 303),
 ('N Queens Blvd', 290),
 ('E Riverside Dr', 277),
 ('W 1st Ave', 276),
 ('S W 52nd St', 272),
 ('W Amsterdam Ave', 264),
 ('E Pearl St', 260),
 ('E Amsterdam Ave', 255),
 ('E Madison Ave', 250),
 ('N W 41st St', 248),
 ('S E 41st St', 242),
 ('N E 41st St', 239),
 ('N W 50th St', 236),
 ('E Water St', 226),
 ('S E 49th St', 225),
 ('N E 54th St', 224),
 ('E 6th Ave', 218),
 ('N E 46th St', 216),
 ('N 89th Ave', 214),
 ('S 89th Ave', 213),
 ('W Riverside Dr', 207),
 ('E W 10th St', 204),
 ('N W 51st St', 203),
 ('N 37th Ave', 202),
 ('E 10th Ave', 201),
 ('N W 52nd St', 200),
 ('W 9th Ave', 197),
 ('E 8th Ave', 197),
 ('N W 40t

sincwe the api is limited by taking it 14 times, so we are using 14 times and jumping loop point to switch apis, hence we can source all the longitute and latitute of the spot

In [25]:
q =[]
count =[]
for i in range(0,166,14):
    if (i//14)%2==0:
        from geopy.geocoders import Nominatim
        for j in range(i,i+14):
            try:
                geolocator =  Nominatim(user_agent="http")
                location = geolocator.geocode(str(add1[j][0]))
                q.append((location.latitude, location.longitude))
                count.append(add1[j][1])
            except:
                pass
    else:
        from geopy.geocoders import Nominatim
        for j in range(i,i+14):
            try:
                geolocator =  Nominatim(user_agent="my_request")
                location = geolocator.geocode(str(add1[j][0]))
                q.append((location.latitude, location.longitude))
                count.append(add1[j][1])
            except:
                pass

In [26]:
len(q)

130

In [28]:
q[0]

(40.7205041, -74.0053416)

In [30]:
count[0]

714

In [32]:
map = folium.Map(location=q[0])
for i in range(len(q)):
    point = q[i]
    rad = math.log(count[i])
    folium.CircleMarker(location=point,radius=rad,color="red", fill=True,fill_color="red",).add_to(map)
map

# Vehicle Collision Data

In [33]:
collisions = pd.read_csv("Motor_Vehicle_Collisions_-_Crashes.csv")
collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [34]:
cs=[]
la=list(collisions['LATITUDE'].fillna(np.nan).dropna())
lo=list(collisions['LONGITUDE'].fillna(np.nan).dropna())
for i in range(len(lo)):
  s= [la[i],lo[i]]
  cs.append(s)

save the longitude and latitudew to txt file

In [36]:
with open('collision.txt', 'w') as writefile:
  writefile.write(str(cs[0]))
  writefile.write("\n")
for i in range(1,len(cs)):
    with open('collision.txt', 'a') as writefile:
      writefile.write(str(cs[i]))
      writefile.write("\n")

Read the txt file

In [38]:
example1 = "collision.txt"
file1 = open(example1, "r")
with open(example1, "r") as file1:
    FileasList = file1.read().splitlines()

In [39]:
len(FileasList)

1727956

check the frequency

In [41]:
from collections import Counter
location = Counter(FileasList)
l = sorted(location.items(), key=lambda x: x[1], reverse=True)

In [42]:
l.remove(l[0])# remove the most frequency location [0,0]

In [43]:
lalong = pd.DataFrame(l)
lalong.head()

,0,1
0,"[40.861862, -73.91282]",685
1,"[40.608757, -74.038086]",670
2,"[40.696033, -73.98453]",646
3,"[40.8047, -73.91243]",597
4,"[40.6960346, -73.9845292]",587


In [44]:
l[1]

('[40.608757, -74.038086]', 670)

The first location( most frequency )has car collision happend 670 times

In [45]:
print(l[1716],l[1717]) 

('[40.7398611, -73.9950165]', 100) ('[40.698463, -73.960205]', 99)


First 1716 places have appeared car collision more than 100 times in 2022

In [46]:
# the maximum capicity of location points of the map is 40000, checking the frequency of the point 4000
print(ast.literal_eval(l[40000][0]),l[40000][1])

[40.6946728, -73.9311439] 10


The places happened car collision more than 100 times

In [47]:
map0 = folium.Map(location=ast.literal_eval(l[0][0]))
for i in range(1716):
  point = ast.literal_eval(l[i][0])
  rad = math.log(10*l[i][1])
  folium.CircleMarker(location=point,radius=rad,color="#3186cc", fill=True,fill_color="#3186cc",).add_to(map0)
map0

Combine the parking violation and the collision map together:

### Red circle is parking violation and blue one is the car colission
### Circle size is taking the log of the frequency times.
- There are 1716 data of the parking violation which has appeared in the same spot more than 100 times.
- there are 166 data of the car colission which happened in the same spot more than 100 times.
### The map below is a combination of both.

In [48]:
map_2 = folium.Map(location=ast.literal_eval(l[0][0]))
for i in range(1716):
  point = ast.literal_eval(l[i][0])
  rad = math.log(10*l[i][1])
  folium.CircleMarker(location=point,radius=rad,color="#3186cc", fill=True,fill_color="#3186cc",).add_to(map_2)
for i in range(len(q)):
    point = q[i]
    rad = math.log(count[i])
    folium.CircleMarker(location=point,radius=rad,color="red", fill=True,fill_color="red",).add_to(map_2)
map_2